In [1]:
import shutil
from pathlib import Path

gastrovision_big_demo_path = Path("demos/gastrovision_big")

if gastrovision_big_demo_path.exists():
    shutil.rmtree(gastrovision_big_demo_path)

gastrovision_big_demo_path.mkdir(parents=True, exist_ok=True)

gastrovision_big_examples_path = gastrovision_big_demo_path / "examples"
gastrovision_big_examples_path.mkdir(parents=True, exist_ok=True)

In [2]:
import torchvision
from torch import nn

def create_effnet2_model(num_classes: int=3):
    weights = torchvision.models.EfficientNet_B2_Weights.DEFAULT
    transforms = weights.transforms()
    model = torchvision.models.efficientnet_b2(weights=None)

    for param in model.parameters():
        param.requires_grad = False

    model.classifier = nn.Sequential(
        nn.Dropout(p=0.3, inplace=True),
        nn.Linear(in_features=1408, out_features=num_classes)
    )

    return model, transforms

In [3]:
import sys
import torch
import random
import shutil
from pathlib import Path
from modules import utils
from safetensors.torch import load_file

venv_dir = Path(sys.prefix)
project_root = venv_dir.parent
dataset_path = project_root/"data"/"food-101"

test_data_paths = list(Path(dataset_path/"images").glob("*/*.jpg"))
example_list = [filepath for filepath in random.sample(test_data_paths, k=15)]
print(example_list)


with open(dataset_path/"meta"/"classes.txt", "r") as f:
    class_names = [line.strip() for line in f if line.strip()]


model, transforms = create_effnet2_model(num_classes=101)
weights_path = "/home/glauco/Desktop/projects/learningPyTorch/trained_models/effnetb2_food101.safetensors"
sd = load_file(weights_path)
model.load_state_dict(sd, strict=True)
model.eval()

results = utils.pred_and_store(
    model=model,
    paths=example_list,
    transforms=transforms,
    class_names=class_names,
    device=torch.device("cpu")
)

print(results)


good_examples = [path for i, path in enumerate(example_list) if results[i]["correct"] == True]
print(good_examples)



for example in good_examples:
    destination = gastrovision_big_examples_path / example.name
    shutil.copy2(src=example, dst=destination)

[PosixPath('/home/glauco/Desktop/projects/learningPyTorch/data/food-101/images/hot_and_sour_soup/1277295.jpg'), PosixPath('/home/glauco/Desktop/projects/learningPyTorch/data/food-101/images/beet_salad/1279787.jpg'), PosixPath('/home/glauco/Desktop/projects/learningPyTorch/data/food-101/images/waffles/1371262.jpg'), PosixPath('/home/glauco/Desktop/projects/learningPyTorch/data/food-101/images/caesar_salad/2970338.jpg'), PosixPath('/home/glauco/Desktop/projects/learningPyTorch/data/food-101/images/club_sandwich/3620923.jpg'), PosixPath('/home/glauco/Desktop/projects/learningPyTorch/data/food-101/images/beef_carpaccio/3574772.jpg'), PosixPath('/home/glauco/Desktop/projects/learningPyTorch/data/food-101/images/greek_salad/1694440.jpg'), PosixPath('/home/glauco/Desktop/projects/learningPyTorch/data/food-101/images/lobster_roll_sandwich/2192375.jpg'), PosixPath('/home/glauco/Desktop/projects/learningPyTorch/data/food-101/images/miso_soup/168469.jpg'), PosixPath('/home/glauco/Desktop/projects

In [5]:
%%writefile demos/gastrovision_big/model.py
import torchvision

from torch import nn

def create_effnet2_model(num_classes: int=3):
    weights = torchvision.models.EfficientNet_B2_Weights.DEFAULT
    transforms = weights.transforms()
    model = torchvision.models.efficientnet_b2(weights=weights)

    for param in model.parameters():
        param.requires_grad = False

    model.classifier = nn.Sequential(
        nn.Dropout(p=0.3, inplace=True),
        nn.Linear(in_features=1408, out_features=num_classes)
    )

    return model, transforms

Writing demos/gastrovision_big/model.py


In [8]:
%%writefile demos/gastrovision_big/app.py
import os
import torch
import gradio as gr

from typing import Tuple, Dict
from model import create_effnet2_model
from timeit import default_timer as timer
from safetensors.torch import load_file, safe_open
from huggingface_hub import hf_hub_download, snapshot_download

class_names = ["pizza", "steak", "sushi"]

effnetb2, effnetb2_transforms = create_effnet2_model(num_classes=101)

MODEL_REPO = "glaucomasi/gastrovision_mini_effnetb2_20percent"
FILENAME   = "09_pretrained_effnetb2_20_percent.safetensors"
weights_path = "/home/glauco/Desktop/projects/learningPyTorch/trained_models/effnetb2_food101.safetensors"

# weights_path = hf_hub_download(
#     repo_id=MODEL_REPO,
#     filename=FILENAME,
#     local_dir=".",                 # optional: place a real copy into your app folder
#     local_dir_use_symlinks=False,  # real file instead of symlink (handy for debug)
# )



print("== Debug: file existence ==")
print("Exists?", os.path.exists(weights_path))
if os.path.exists(weights_path):
    print("File size (bytes):", os.path.getsize(weights_path))
    # Peek first bytes
    with open(weights_path, "rb") as f:
        head = f.read(200)
    print("First 200 bytes:", head[:200])
else:
    print("File not found at:", weights_path)

# Try safetensors open
try:
    with safe_open(weights_path, framework="pt") as f:
        keys = list(f.keys())
        print("== Debug: safetensors keys sample ==")
        print(keys[:10])
except Exception as e:
    print("== Debug: safetensors load error ==")
    print(repr(e))

sd = load_file(weights_path)
effnetb2.load_state_dict(sd, strict=True)
effnetb2.eval()



def predict(img) -> Tuple[Dict, float]:
    start_time = timer()

    img = effnetb2_transforms(img).unsqueeze(dim=0)

    effnetb2.eval()
    with torch.inference_mode():
        pred_probs = torch.softmax(effnetb2(img), dim=1)

    # Gradio's required format: a dictionary with the probability for each class
    pred_labels_and_probs = {class_names[i]: float(pred_probs[0][i]) for i in range(len(class_names))}
    pred_time = round(timer()-start_time, 5)

    return pred_labels_and_probs, pred_time



title = "Food101 Vision Model"
description = "An EfficientNetB2 feature extractor computer vision model achieving '65%' accuracy on Food101 test dataset"
article = "Created by Glauco Masi"

example_list = [["examples/"+example] for example in os.listdir("examples")]

demo = gr.Interface(
    fn=predict,
    inputs=gr.Image(type="pil"),
    outputs=[
        gr.Label(num_top_classes=3, label="Predictions"),
        gr.Number(label="Prediction time (s)")
    ],
    examples=example_list,
    title=title,
    description=description,
    article=article
)

demo.launch()

Writing demos/gastrovision_big/app.py


In [ ]:
import torch
import gradio
import torchvision
import safetensors
print(torch.__version__)
print(torchvision.__version__)
print(gradio.__version__)
print(safetensors.__version__)

2.8.0+cu128
0.23.0+cu128
5.44.1
0.6.2


In [6]:
%%writefile demos/gastrovision_big/requirements.txt
torch==2.8.0
torchvision==0.23.0
gradio==5.44.1
safetensors==0.6.2

Writing demos/gastrovision_big/requirements.txt
